In [1]:
import pandas as pd
import numpy as np
import string

In [39]:
filename = "/Users/ahmedabukar/Downloads/:/data/winemag-150k-reviews.csv"
df = pd.read_csv(filename,
                usecols=["country", "province", "description", "variety"])
df.head()

,country,description,province,variety
0,US,This tremendous 100% varietal wine hails from ...,California,Cabernet Sauvignon
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Northern Spain,Tinta de Toro
2,US,Mac Watson honors the memory of a wine once ma...,California,Sauvignon Blanc
3,US,"This spent 20 months in 30% new French oak, an...",Oregon,Pinot Noir
4,France,"This is the top wine from La Bégude, named aft...",Provence,Provence red blend


In [12]:
def remove_words(df_):
    return (df_["description"]
    .str.split()
    .explode()
    .str.strip(string.punctuation)
    .str.lower()
    .loc
            [
            lambda s_ : ~s_.isin(["flavors", "aromas", "finish", "palate", "drink"])
                & (s_.str.len() >= 5)
            ]
            .value_counts()
            .head(10)
           )

In [4]:
# what are the 10 most common words containing five or more letters in the wine description?
#turn all words into lower case`
# remove all punctuation and symbols at the end and begining of each word
# also remove the words 'flavors', "aromas", 'finish', 'palate', and 'drink'
wine_words = (
    df["description"].str.lower()
 .str.split()
    .explode()
    .str.strip(string.punctuation)
)

(
    wine_words
    [(~wine_words.isin(['flavors', "aromas", "finish", "palate", "drink"])) 
    & (wine_words.str.len() >= 5)]
    .value_counts()
    .head(10)
)

description
fruit      56496
acidity    32603
tannins    32186
cherry     30685
black      24599
spice      22658
sweet      21284
notes      19606
fresh      17662
berry      17104
Name: count, dtype: int64

In [14]:
# what are the 10 most common words for non-californian wines?
(
 df
 .loc[df["province"] != "California"]
 .pipe(remove_words)
)


description
fruit      46517
acidity    22325
tannins    22007
cherry     19473
spice      18572
black      17785
notes      16591
fresh      16221
berry      15499
sweet      12741
Name: count, dtype: int64

In [15]:
# what are the 10 most common words for french wines?
(
    df
    .loc[df["country"] == "France"]
    .pipe(remove_words)
)


description
fruit        8722
acidity      8641
tannins      6509
fruits       5459
fresh        4217
character    3499
black        3124
texture      3074
years        2957
crisp        2879
Name: count, dtype: int64

In [16]:
# what are the 10 most common words for white wines?
# white wines are : "Chardonnay", "Sauvignon Blanc", "Riesling"
(
    df.loc[df["variety"].isin(["Chardonnay", "Sauvignon Blanc", "Riesling"])]
    .pipe(remove_words)
)


description
fruit         9156
acidity       8367
apple         5893
citrus        5384
chardonnay    4914
crisp         4908
green         4191
notes         4030
pineapple     3859
sweet         3859
Name: count, dtype: int64

In [17]:
# what are the 10 most common words for red wines? 
# for purposes look for: "Pinot Noir", "Carbinet Sauvignon", "Syrah", "Merlot" and "Zinfandel"
(
    df
    .loc[
    df["variety"]
    .isin(["Pinot Noir", "Carbernet Sauvignon", 'Syrah', "Merlot", "Zinfandel"])
    ]
    .pipe(remove_words)
    
)


description
fruit        10558
cherry       10525
tannins       8034
pinot         6322
black         5728
acidity       5231
cherries      4254
sweet         3979
spice         3853
raspberry     3780
Name: count, dtype: int64

In [18]:
# what are the 10 most common words for rose wines?
(
    df.loc[df["variety"] == "Rosé"]
    .pipe(remove_words)
)

description
acidity       1135
fruit          697
crisp          672
fresh          622
strawberry     534
light          518
raspberry      510
cherry         470
fruity         428
fruits         420
Name: count, dtype: int64

In [35]:
# show the 10 most common words for the five most common wine varieties?
(
df
  .loc[
     df["variety"]
        .isin(df["variety"].value_counts().head().index)
    ]
    .pipe(remove_words)
)

description
fruit       22859
tannins     16007
cherry      13991
acidity     12526
black       11236
cabernet     9480
spice        7910
sweet        7881
blend        7597
shows        7264
Name: count, dtype: int64

In [56]:
# which country's wines has got the highest average score?
df1 = pd.read_csv(filename, 
                 usecols=["country", "points", "description", "province", "variety"])
(
    df1
      .groupby("country")
       ["points"]
        .mean()
        .sort_values()
         .tail(1)
)

country
England    92.888889
Name: points, dtype: float64

In [47]:
# create a pivot table in which the index contains countries, the columns contains the varieties,
# and the cells contains the mean scores. Include only the top 10 varieties.
(
    df1
    .loc[
     df["variety"]
        .isin(df["variety"].value_counts().head(10).index)
    ]
    .pivot_table(
        index="country",
        columns="variety",
        values="points"
    )
)

variety,Bordeaux-style Red Blend,Cabernet Sauvignon,Chardonnay,Merlot,Pinot Noir,Red Blend,Riesling,Sauvignon Blanc,Syrah,Zinfandel
country,,,,,,,,,,
Argentina,89.575472,85.527745,84.177489,84.341969,85.058333,88.197059,85.000000,83.295302,85.232394,NaN
Australia,88.841463,88.115502,86.727952,85.258824,86.405263,87.816176,87.790210,86.624060,91.952381,88.200000
Austria,91.625000,87.750000,90.016393,89.000000,88.753846,88.890511,90.583955,88.694215,87.000000,NaN
Brazil,86.000000,81.000000,NaN,83.200000,NaN,84.000000,NaN,NaN,NaN,NaN
Bulgaria,NaN,84.812500,86.875000,84.363636,87.400000,89.000000,83.750000,84.400000,90.000000,NaN
Canada,89.000000,88.666667,88.653846,87.875000,89.111111,89.500000,87.564516,87.750000,89.666667,NaN
Chile,89.754717,86.561564,85.246011,84.939189,85.827273,88.683168,85.714286,85.895805,87.506739,85.000000
China,NaN,NaN,82.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Croatia,NaN,NaN,85.000000,NaN,84.000000,86.000000,NaN,82.000000,NaN,NaN


In [76]:
# what is the number of wines of each country?
number_of_wines = df1.groupby("country")["variety"].count().sort_values()

mean_points = df1.groupby("country")["points"].mean().sort_values()

number_of_wines.to_frame().join(mean_points)

,variety,points
country,,
US-France,1,88.000000
Albania,2,88.000000
Tunisia,2,86.000000
Montenegro,2,82.000000
Japan,2,85.000000
Slovakia,3,83.666667
China,3,82.000000
Egypt,3,83.666667
Switzerland,4,87.250000


In [75]:
df1.groupby("country")["points"].agg(["count", "mean"])

,count,mean
country,,
Albania,2,88.000000
Argentina,5631,85.996093
Australia,4957,87.892475
Austria,3057,89.276742
Bosnia and Herzegovina,4,84.750000
Brazil,25,83.240000
Bulgaria,77,85.467532
Canada,196,88.239796
Chile,5816,86.296768
